In [15]:
%load_ext autoreload
%autoreload 2
#%pylab inline

In [16]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [17]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

In [34]:
model = keras.models.load_model('models/model_20170209-171440/model_epoch99.h5')

In [35]:
env = AtariStackFrames(gym.make('MsPacman-v0'), rgb=True)

[2017-02-09 19:33:26,203] Making new env: MsPacman-v0


In [36]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        #action = env.action_space.sample()
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [37]:
res = []
for i in xrange(20):
    r = run_episode(env, model)
    print r
    res.append(r)
print 'mean: {:.3f}'.format(np.array(res).mean())

610.0
1020.0
310.0
320.0
590.0
400.0
780.0
560.0
600.0
1100.0
300.0
340.0
640.0
540.0
440.0
450.0
850.0
2290.0
1150.0
720.0
mean: 700.500


## temp

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [ ]:
inp = Input(shape=(1,))
out = Dense(32, activation='relu')(inp)
out = Dense(1)(out)

In [ ]:
model = Model(input=inp, output=out)

In [ ]:
model.save('temp.h5')

In [3]:
m = keras.models.load_model('temp.h5')
for var in m.weights:
    print var.op.name

dense_1_W
dense_1_b
dense_2_W
dense_2_b


/home/user/.local/lib/python2.7/site-packages/keras/models.py:150: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
with tf.variable_scope('policy11'):
    m = keras.models.load_model('temp.h5')
    for var in m.weights:
        print var.op.name

In [4]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 32)            64          input_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             33          dense_1[0][0]                    
Total params: 97
Trainable params: 97
Non-trainable params: 0
____________________________________________________________________________________________________


In [7]:
m.layers[-2].output

<tf.Tensor 'Relu:0' shape=(?, 32) dtype=float32>

In [6]:
m.input

<tf.Tensor 'input_1:0' shape=(?, 1) dtype=float32>

In [10]:
with tf.variable_scope('policy'):
    out = Dense(128)(m.layers[-2].output)
    new = Model(input=m.layers[0].input, output=out)

In [12]:
new.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 32)            64          input_1[0][0]                    
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 128)           4224        dense_1[0][0]                    
Total params: 4,288
Trainable params: 4,288
Non-trainable params: 0
____________________________________________________________________________________________________


In [14]:
for var in new.weights:
    print var.op.name

dense_1_W
dense_1_b
policy_3/dense_4_W
policy_3/dense_4_b


In [11]:
m.layers[0].input

<tf.Tensor 'input_1:0' shape=(?, 1) dtype=float32>